In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/Users/cslage/Research/LSST/code/GUI/cte/20161028_029-cte-timing/'
%cd $thedir

#%matplotlib inline

In [ ]:
files = sort(glob.glob('ITL-3800C-029_dark_dark_0??_20161028??????_bias.fits'))
print files
name = files[0][24:27]
print name

In [ ]:
# [seg, xstart, xpeak, ystart, ystop]
regions = [[8,190,197,754,756],[8,190,197,765,768],[11,490,500,375,386],[11,490,500,402,411]]
figure()
subplots_adjust(hspace = 0.5, wspace = 0.5)
suptitle("CTI - Hot Pixels", fontsize = 16)
plotcounter = 1
for file in files:
    name = file[24:27]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    for [segment,xstart,xpeak,ystart,ystop] in regions:
        xstop = xstart+20
        xaxis = linspace(xstart,xstop-1,20)
        subplot(2,2,plotcounter)
        plotcounter += 1
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("%s, x = %d, y = %d"%(extname,xpeak,(int((ystart+ystop)/2))), fontsize = 10)
        data = array(hdulist[segment+1].data + 32768, dtype = int32)
        cte_data = data[ystart:ystop,:].mean(axis=0)[xstart:xstop]
        cte_std = (data[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
        ratio = data[ystart:ystop,xpeak+1].mean() / data[ystart:ystop,xpeak].mean()
        cti = ratio / xpeak
        scatter(xaxis, cte_data, color = 'blue')
        errorbar(xaxis, cte_data, yerr=cte_std)
        text(xstart-2, 20000, "Ratio = %.3E"%ratio,fontsize=6)
        text(xstart-2, 10000, "CTI = %.3E"%cti,fontsize=6)
        text(xstart,100,"<----",color = 'blue')
        yscale('log',nonposy='clip')
        ylim(.1, 100000)
        xlabel("X")
        ylabel("Signal(ADU)")
savefig("Serial_CTE_Hot_Pixels_%s_28Oct16.pdf"%name)


In [ ]:
regions = [[8,190,197,754,756],[8,190,197,765,768],[11,490,500,375,386],[11,490,500,402,411]]
figure()
subplots_adjust(hspace = 0.5, wspace = 0.5)
suptitle("CTI - Hot Pixels", fontsize = 12)
delays = [450, 950, 1450]
ratios = zeros([4,3])
colors = ['blue', 'red', 'green']
for i, file in enumerate(files):
    name = file[24:27]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    for j,[segment,xstart,xpeak,ystart,ystop] in enumerate(regions):
        xstop = xstart+20
        xaxis = linspace(xstart,xstop-1,20)
        subplot(2,2,j+1)
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("%s, x = %d, y = %d"%(extname,xpeak,(int((ystart+ystop)/2))), fontsize = 9)
        data = array(hdulist[segment+1].data + 32768, dtype = int32)
        cte_data = data[ystart:ystop,:].mean(axis=0)[xstart:xstop]
        cte_std = (data[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
        ratio = data[ystart:ystop,xpeak+1].mean() / data[ystart:ystop,xpeak].mean()
        ratios[j,i] = ratio
        cti = ratio / xpeak
        scatter(xaxis, cte_data, color = colors[i], label = "%dns"%delays[i])
        errorbar(xaxis, cte_data, yerr=cte_std, color = colors[i])
        #text(xstart-2, 20000, "Ratio = %.3E"%ratio,fontsize=6)
        #text(xstart-2, 10000, "CTI = %.3E"%cti,fontsize=6)
        text(xstart,100,"<----",color = 'blue')
        yscale('log',nonposy='clip')
        ylim(.1, 100000)
        xlabel("X")
        ylabel("Signal(ADU)")
        legend(prop={'size':9}, bbox_to_anchor=(0.6, 1), loc=2, borderaxespad=0.)
savefig("Serial_CTE_Hot_Pixels_Delays_25Apr18.pdf")

figure()
subplots_adjust(hspace = 0.5, wspace = 0.5)
suptitle("Pixel ratio vs Delay")
for j,[segment,xstart,xpeak,ystart,ystop] in enumerate(regions):
    hdulist = pf.open(files[0], mode='readonly', do_not_scale_image_data=True)
    imhdr=hdulist[segment+1].header
    extname = imhdr['EXTNAME']
    subplot(2,2,j+1)
    title("%s, x = %d, y = %d"%(extname,xpeak,(int((ystart+ystop)/2))), fontsize = 9)
    scatter(delays, ratios[j,:])
    slope, intercept, r_value, p_value, std_err = stats.linregress(delays, log(ratios[j,:]))
    xplot=linspace(0,2000,100)
    yplot = exp(intercept + slope * xplot)
    plot(xplot, yplot, ls = '--', color = 'red')
    text(100, 2E-4, "Tau = %.1f nanoseconds"%(-1.0/slope))
    yscale('log')
    ylim(1.0E-4,1.0E-2)
    xlim(0, 2000)
    ylabel("Ratio - First Trailing Pixel / Peak", fontsize = 10)
    xlabel("Delay(nanoseconds)", fontsize = 10)
savefig("CTI_Delay_Hot_Pixels_25Apr18.pdf")
